In [2]:
import os
import json
import argparse
import re
from tqdm import tqdm
import jieba  # 用於中文文本分詞
import pdfplumber  # 用於從PDF文件中提取文字的工具
from rank_bm25 import BM25Okapi  # 使用BM25演算法進行文件檢索
from openai import OpenAI

In [2]:
# 載入參考資料，返回一個字典，key為檔案名稱，value為PDF檔內容的文本
def load_data(source_path):
    masked_file_ls = os.listdir(source_path)  # 獲取資料夾中的檔案列表
    corpus_dict = {int(file.replace('.pdf', '')): read_pdf(os.path.join(source_path, file)) for file in tqdm(masked_file_ls)}  # 讀取每個PDF文件的文本，並以檔案名作為鍵，文本內容作為值存入字典
    return corpus_dict

In [3]:
# 讀取單個PDF文件並返回其文本內容
max_tokens_per_request = 4000
def read_pdf(pdf_loc, page_infos: list = None):
    pdf = pdfplumber.open(pdf_loc)  # 打開指定的PDF文件
    client = OpenAI()
    # TODO: 可自行用其他方法讀入資料，或是對pdf中多模態資料（表格,圖片等）進行處理

    # 如果指定了頁面範圍，則只提取該範圍的頁面，否則提取所有頁面
    pages = pdf.pages[page_infos[0]:page_infos[1]] if page_infos else pdf.pages
    pdf_text = ''
    for _, page in enumerate(pages):  # 迴圈遍歷每一頁
        text = page.extract_text()  # 提取頁面的文本內容
        if text:
            text = text.replace('\n', '').replace('\r', '').replace(' ', '').replace(',', '').replace('-', '').replace('%', '').replace('~', '').replace('.', '').replace('民國', '').replace('一', '').replace('二', '').replace('三', '').replace('四', '').replace('五', '').replace('六', '').replace('七', '').replace('八', '').replace('九', '').replace('十', '').replace('零', '').replace('年', '').replace('月', '').replace('日', '').replace('至', '').replace('、','').replace('$','').replace('～','')
            text = re.sub(r'\d+', '', text)
            text = re.sub(r'\(.*?\)', '', text)
            text = re.sub(r'\（.*?\）', '', text)
            pdf_text += text
    pdf_text_segments = [pdf_text[i:i+max_tokens_per_request] for i in range(0, len(pdf_text), max_tokens_per_request)]
    responses = []
    for segment in pdf_text_segments:
        completion =client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {
                    "role": "user",
                    "content": (
                        "我會提供給你一段文字，是用python的pdf OCR辨認出來的文字。"
                        "因為pdf中會有不規則的表格，所以有時候會有中文然後突然出現一些不規則的英文或數字。"
                        "請幫我把後面的文章的每一句話都整理成重點而且看得懂的句子。"
                        "有些句子內會含有多個財報或公司的相關名詞，也請都幫我保留。"
                        "除了我請你幫我整理的答案以外請不要輸出其他任何東西。" + segment
                    )
                }
            ]
        )
        # 收集每個片段的返回結果
        responses.append(completion.choices[0].message.content)

    # 將所有片段的結果合併成一個字符串返回
    return ''.join(responses)

In [4]:
# 根據查詢語句和指定的來源，檢索答案
def BM25_retrieve(qs, source, corpus_dict):
    filtered_corpus = [corpus_dict[int(file)] for file in source]
    # [TODO] 可自行替換其他檢索方式，以提升效能
    tokenized_corpus = [list(jieba.cut_for_search(doc)) for doc in filtered_corpus]  # 將每篇文檔進行分詞
    bm25 = BM25Okapi(tokenized_corpus)  # 使用BM25演算法建立檢索模型
    tokenized_query = list(jieba.cut_for_search(qs))  # 將查詢語句進行分詞
    ans = bm25.get_top_n(tokenized_query, list(filtered_corpus), n=1)  # 根據查詢語句檢索，返回最相關的文檔，其中n為可調整項
    a = ans[0]
    # 找回與最佳匹配文本相對應的檔案名
    res = [key for key, value in corpus_dict.items() if value == a]
    return res[0]  # 回傳檔案名

In [5]:
import sys

# 模擬命令行參數
sys.argv = [
    "notebook",
    "--question_path", "/Users/yhk/Desktop/大三上/DataMining/FP/競賽資料集/dataset/preliminary/questions_example.json",
    "--source_path", "/Users/yhk/Desktop/大三上/DataMining/FP/競賽資料集/reference",
    "--output_path", "/Users/yhk/Desktop/大三上/DataMining/FP/競賽資料集/dataset/preliminary/pred_retrieve.json",
]

In [6]:
if __name__ == "__main__":
    # 使用argparse解析命令列參數
    parser = argparse.ArgumentParser(description='Process some paths and files.')
    parser.add_argument('--question_path', type=str, required=True, help='讀取發布題目路徑')  # 問題文件的路徑
    parser.add_argument('--source_path', type=str, required=True, help='讀取參考資料路徑')  # 參考資料的路徑
    parser.add_argument('--output_path', type=str, required=True, help='輸出符合參賽格式的答案路徑')  # 答案輸出的路徑
    
    args = parser.parse_args()  # 解析參數

    answer_dict = {"answers": []}  # 初始化字典

    with open(args.question_path, 'rb') as f:
        qs_ref = json.load(f)  # 讀取問題檔案

    source_path_insurance = os.path.join(args.source_path, 'insurance')  # 設定參考資料路徑
    corpus_dict_insurance = load_data(source_path_insurance)

    source_path_finance = os.path.join(args.source_path, 'finance')  # 設定參考資料路徑
    corpus_dict_finance = load_data(source_path_finance)

    with open(os.path.join(args.source_path, 'faq/pid_map_content.json'), 'rb') as f_s:
        key_to_source_dict = json.load(f_s)  # 讀取參考資料文件
        key_to_source_dict = {int(key): value for key, value in key_to_source_dict.items()}

    for q_dict in qs_ref['questions']:
        if q_dict['category'] == 'finance':
            # 進行檢索
            retrieved = BM25_retrieve(q_dict['query'], q_dict['source'], corpus_dict_finance)
            # 將結果加入字典
            answer_dict['answers'].append({"qid": q_dict['qid'], "retrieve": retrieved, "category": "finance"})

        elif q_dict['category'] == 'insurance':
            retrieved = BM25_retrieve(q_dict['query'], q_dict['source'], corpus_dict_insurance)
            answer_dict['answers'].append({"qid": q_dict['qid'], "retrieve": retrieved, "category": "insurance"})

        elif q_dict['category'] == 'faq':
            corpus_dict_faq = {key: str(value) for key, value in key_to_source_dict.items() if key in q_dict['source']}
            retrieved = BM25_retrieve(q_dict['query'], q_dict['source'], corpus_dict_faq)
            answer_dict['answers'].append({"qid": q_dict['qid'], "retrieve": retrieved, "category": "faq"})

        else:
            raise ValueError("Something went wrong")  # 如果過程有問題，拋出錯誤

    with open(args.output_path, 'w', encoding='utf8') as f:
        json.dump(answer_dict, f, ensure_ascii=False, indent=4)  # 儲存檔案，確保格式和非ASCII字符


100%|██████████| 1035/1035 [1:57:53<00:00,  6.83s/it]   
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wt/6hl8qhss3jz9zrdh6n8f3_qh0000gn/T/jieba.cache
Loading model cost 0.314 seconds.
Prefix dict has been built successfully.


In [7]:
corpus_dict_finance

{570: '1. 處分不動產之金額未達新台幣億元或實收資本額百分之以上。\n2. 與關係人進銷貨之金額達新台幣億元或實收資本額百分之以上，詳見附表。\n3. 應收關係人款項達新台幣億元或實收資本額百分之以上，詳見附表。\n4. 無從事衍生工具交易情事。\n5. 母公司與子公司及各子公司間之業務關係及重要交易往來情形及金額，詳見附表。\n6. 轉投資事業相關資訊，包括被投資公司名稱、所在地區等，詳見附表。\n7. 大陸投資資訊基本資料，詳見附表。\n8. 直接或間接經由第地區事業與轉投資大陸之被投資公司發生之重大交易事項無此情事。\n9. 主要股東資訊，詳見附表。\n10. 合併公司已依據營運決策者報導資訊辨認應報導部門，部門之劃分基礎於本期並無重大改變。\n11. 部門損益調節資訊，部門間銷售按公允交易原則進行。\n12. 向主要營運決策者呈報的外部收入與損益表內收入採用一致的衡量方式。\n13. 提供主要營運決策者之總資產金額，與合併公司財務報表內之資產採用一致的衡量方式。',
 216: '1. 光寶科技股份有限公司及其子公司的合併權益變動表中，權益以新台幣仟元為單位，除非另有說明。\n2. 權益項目包括母公司業主的權益、其他權益及其他綜合損益。\n3. 涉及的項目包括股本、資本公積、保留盈餘及各類金融資產。\n4. 其他權益項目中，列出了非控制權益、庫藏股票、待註銷股本、法定盈餘公積及特別盈餘公積。\n5. 計算的總和中包括未分配盈餘及兌換差額。\n6. 殘餘的權益總額計算包括：權益分配、現金股利、非控制權益變動及淨利。\n7. 採用權益法認列的關聯企業及合資變動數被列入。\n8. 資本公積及庫藏股的調整也被列為權益變動的項目。\n9. 其他綜合利益及綜合利益總額同樣被納入計算。\n10. 本合併財務報告附註由董事長宋明峰、經理人邱森彬及會計主管蕭庭宇簽署確認。',
 202: '1. 中國鋼鐵股份有限公司設立於特定年份，並分階段完成建廠計畫，主要從事鋼鐵製品的製造與銷售，以及相關工程的承攬。\n2. 本公司及子公司如中國鋼鐵結構公司、中鋼碳素化學公司、中聯資源公司、中宇環保工程公司和中鴻鋼鐵公司等，已在台灣證券交易所上市。\n3. 子公司鑫科材料科技公司已在中華證券櫃檯買賣中心掛牌交易。\n4. 中龍鋼鐵公司已辦理股票公開發行。\n5. 截止至某一日期，經

In [8]:
corpus_dict_finance[628]

'1. 子公司和泰產險已簽約的工程價款為尚未支付的金額。\n2. 本公司與關係人及非關係人所簽訂的重要契約彙總中，無重大災害損失。\n3. 子公司和潤企業在董事會決議通過多項事項，包括代製約及其他合約。\n4. 擬籌組銀行聯貸案，金額預計為人民幣千元至千元之間。\n5. 擬籌組海外聯貸案，金額預計為貨幣千元至千元之間。\n6. 擬籌組海外借款案，金額預計為美金千元。\n7. 擬發行無擔保及有擔保的普通公司債，金額預計為某金額。\n8. 子公司和運國際租賃有限公司將籌組銀行聯貸案，金額預計為人民幣千元至千元之間。\n9. 子公司和泰興業股份有限公司標購桃園市沙崙產業園區廠房用地，交易總金額未詳述。\n10. 本集團的資本管理目標是保障經營持續性，考量未來資金需求及長期規劃，維持最佳資本結構以降低資金成本，支持企業營運及股東權益。\n11. 子公司和泰產險已依「保險法」制定資本管理政策。\n12. 金融工具種類包括透過損益按公允價值衡量的金融資產，以及其他現金及應收款項等。'

In [9]:
sys.argv = [
    "notebook",
    "--true_answer_path", "/Users/yhk/Desktop/大三上/DataMining/FP/競賽資料集/dataset/preliminary/ground_truths_example.json",
    "--pred_answer_path", "/Users/yhk/Desktop/大三上/DataMining/FP/競賽資料集/dataset/preliminary/pred_retrieve.json"
]
parser = argparse.ArgumentParser(description='Process some paths and files.')
parser.add_argument('--true_answer_path', type=str, required=True, help='真實答案的答案路徑')  # 答案輸出的路徑
parser.add_argument('--pred_answer_path', type=str, required=True, help='輸出符合參賽格式的答案路徑')  # 答案輸出的路徑
args = parser.parse_args()
caterror = {"faq":0, "insurance" :0, "finance" :0}
retrieveerror = {"faq":0, "insurance" :0, "finance" :0}
catelen = {"faq":0, "insurance" :0, "finance" :0}
with open(args.true_answer_path, 'rb') as f:
    true_ans_ref = json.load(f) 
with open(args.pred_answer_path, 'rb') as f:
    pred_ans_ref = json.load(f) 
true_ans_ref = true_ans_ref["ground_truths"]
pred_ans_ref = pred_ans_ref["answers"]
for i in range(len(true_ans_ref)):
    catelen[true_ans_ref[i]['category']] += 1
    if true_ans_ref[i]['category'] != pred_ans_ref[i]['category']:
        caterror[true_ans_ref[i]['category']] += 1
        retrieveerror[true_ans_ref[i]['category']] +=1
    elif true_ans_ref[i]['retrieve'] != pred_ans_ref[i]['retrieve']:
        retrieveerror[true_ans_ref[i]['category']] +=1
print("Error rate")
print("Category Error")
for cate in list(caterror.keys()):
    print(f"{cate}: {caterror[cate]/catelen[cate]}")
print("Retrieve Error")
for cate in list(caterror.keys()):
    print(f"{cate}: {retrieveerror[cate]/catelen[cate]}")





Error rate
Category Error
faq: 0.0
insurance: 0.0
finance: 0.0
Retrieve Error
faq: 0.1
insurance: 0.32
finance: 0.28


In [10]:
list(caterror.keys())

['faq', 'insurance', 'finance']

NameError: name 'OpenAi' is not defined